In [46]:
from turtle import shape
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
import scipy.io as spio
from LLR import LLR
import math

n_ch = 1000
h_ch = np.load(f'H6Length{n_ch}.npy')
ch_len = 5
h_sp = h_ch[ch_len//2][0:ch_len] 

h = np.zeros((ch_len,ch_len))

for i in range(ch_len//2+1):
    h[i][:ch_len//2 + i + 1] = h_sp[ch_len//2 - i:]
for i in range(ch_len//2+1):
    h[ch_len//2+i] = h[ch_len//2-i][::-1]
h

array([[ 0.84867432,  0.34356324, -0.14159457,  0.        ,  0.        ],
       [ 0.34356324,  0.84867432,  0.34356324, -0.14159457,  0.        ],
       [-0.14159457,  0.34356324,  0.84867432,  0.34356324, -0.14159457],
       [ 0.        , -0.14159457,  0.34356324,  0.84867432,  0.34356324],
       [ 0.        ,  0.        , -0.14159457,  0.34356324,  0.84867432]])

In [76]:
class cluster:
    def __init__(self,n_tran=17):
        self.n_tran = n_tran
        self.mid_tran = self.n_tran//2
        self.trun_len = 5
        self.mid_trun = self.trun_len//2
        self.llrVal = []
    
    def fitModel(self,neigh=1):
        pos = []
        neg = []
        # cnt = 0
        points = set()
        X = []
        Y = []
        self.s_in = []
        for i in range(2**self.n_tran):
            temp = 0
            sig = []
            for j in range(self.n_tran):
                if (i&(2**j)) != 0:
                    sig += [1]
                else: 
                    sig += [-1]
            o = []
            Y += [sig[self.mid_tran]]
            for j in range(self.mid_trun,self.n_tran - self.mid_trun):
                o += [np.round(np.dot(sig[j-self.mid_trun:j+self.mid_trun+1],h_sp),4)]
            X += [o]
            self.s_in += [sig[self.mid_trun:self.n_tran-self.mid_trun]]
            points.add(tuple(o)) 
            

            # if sig[self.mid_tran] < 0: 
            #     neg += [o[len(o)//2]]
            # else:
            #     pos += [o[len(o)//2]]
            

            temp = np.round(temp,4)
        

        print("number of cluster points:", len(points))
        self.X_train = np.array(X)
        self.Y_train = np.array(Y)
        pos = np.unique(pos)
        neg = np.unique(neg)
        print(*pos)
        # print(cnt)
        print("-*******-")
        print(*neg)
        self.s_in = np.array(self.s_in)
        
        print('cluster points shape',self.X_train.shape)
        self.model = KNeighborsClassifier(n_neighbors=neigh,weights='distance').fit(self.X_train,self.Y_train)
        # self.model = GaussianNB().fit(X,Y)
        print('error in training:', sum(self.model.predict(self.X_train) != self.Y_train))
        return self.model

    def reduced_model(self):
        
        return
    
    

    def testModelLLR(self,ebno,data):
        noisePw = 1/(10**(ebno/10))
        dim = self.n_tran - self.trun_len + 1
        print(dim)
        self.llrVal = []
        for step in range(0,len(data),n_ch):
            print(step)
            s = 2*data[step:step+n_ch]-1
            s = np.array(s).reshape((n_ch, 1))
            noise = np.sqrt(noisePw/2)*np.random.standard_normal((n_ch,1))
            r = np.dot(h_ch,s) + noise
            X_t = np.zeros((n_ch-(dim-1),dim))
            Y = []
            for i in range(dim//2,n_ch-dim//2):
                for idx,val in enumerate(r[i-dim//2:i+dim//2+1]):
                    X_t[i-dim//2][idx] = np.round(val,4)
                Y += [1 if s[i] > 0 else -1]
            Y = np.array(Y)
            # Y_t = self.model.predict(X_t)
            
            self.X_test = X_t
            Y_t = self.model.kneighbors(self.X_test)
            
#             self.llrVal += [20.0,20.0]
            for j in range(len(Y_t[0])):
                candList = []
                for idx in Y_t[1][j]:
                    candList += [self.s_in[idx].T]
                outputLLR = LLR(n = dim, candList = candList, noisePw=noisePw, dis=Y_t[0][j])
                
                if j == 0: self.llrVal += outputLLR[0][:dim//2+1]
                elif j == len(Y_t[0]) - 1: self.llrVal += [outputLLR[0][dim//2:]]
                else: self.llrVal += [outputLLR[dim//2]]
                
                print(j,len(Y_t[0]),len(self.llrVal),self.llrVal)
            
        return self.llrVal

    def testModel(self,ebno,num_iteration):
        noisePw = 1/(10**(ebno/10))
        dim = self.n_tran - self.trun_len + 1
        err = []
        for _ in range(num_iteration):
            print(_)
            s = 2*np.random.randint(2,size=(n_ch,1))-1
            noise = np.sqrt(noisePw/2)*np.random.standard_normal((n_ch,1))
            r = np.dot(h_ch,s) + noise
            X_t = np.zeros((n_ch-(self.n_tran-1),dim))
            Y = []
            for i in range(self.mid_tran,n_ch-self.mid_tran):
                for id,val in enumerate(r[i-dim//2:i+dim//2+1]):
                    X_t[i-self.mid_tran][id] = np.round(val,4)
                Y += [1 if s[i] > 0 else -1]
            Y = np.array(Y)
            
            Y_t = self.model.predict(X_t)
            err += [sum(Y_t != Y)/len(Y)]
        print(np.mean(err))
        return np.mean(err)


In [117]:
mat = spio.loadmat('./data/testData5e3_int.mat')
dataIn = np.array(mat['intrl'],dtype=int)[0]
print(len(dataIn))

10000


In [83]:
dataIn[:10]

array([1, 0, 1, 0, 1, 0, 1, 1, 0, 1])

In [95]:

sysModel = cluster()
sysModel.fitModel(neigh=8)


number of cluster points: 131072

-*******-

cluster points shape (131072, 13)
error in training: 0


KNeighborsClassifier(n_neighbors=8, weights='distance')

In [118]:

ebno = 4
codeRate = 0.5
snr = ebno + 10*math.log10(codeRate)
print("snr",snr)
noisePw = 1/(10**(snr/10))
dim = sysModel.n_tran - sysModel.trun_len + 1
print(dim)
llrVal = []
err = []
for step in range(0,len(dataIn),n_ch):
    print(step)
    s = 2*dataIn[step:step+n_ch]-1
    s = np.array(s).reshape((n_ch, 1))
    noise = np.sqrt(noisePw/2)*np.random.standard_normal((n_ch,1))
    r = np.dot(h_ch,s) + noise
    X_t = np.zeros((n_ch-(dim-1),dim))
    Y = []
    for i in range(dim//2,n_ch-dim//2):
        for idx,val in enumerate(r[i-dim//2:i+dim//2+1]):
            X_t[i-dim//2][idx] = np.round(val,4)
        Y += [1 if s[i] > 0 else -1]
    Y = np.array(Y)
    # Y_t = self.model.predict(X_t)

    X_test = X_t
    Y_t = sysModel.model.kneighbors(X_test)
    
    Y_tt = sysModel.model.predict(X_t)
    err += [sum(Y_tt != Y)/len(Y)]
    
    
    for j in range(len(Y_t[0])):
        candList = []
        for idx in Y_t[1][j]:
            candList += [sysModel.s_in[idx].T]
        outputLLR = LLR(n = dim, candList = candList, noisePw=noisePw, dis=Y_t[0][j])

        if j == 0: llrVal += list(outputLLR[:dim//2+1])
        elif j == len(Y_t[0]) - 1: llrVal += list(outputLLR[dim//2:])
        else: llrVal += [outputLLR[dim//2]]

print(np.mean(err))
    
print(len(llrVal))

snr 0.9897000433601879
13
0
1000
2000
3000
4000
5000
6000
7000
8000
9000
0.12722672064777327
10000


In [119]:
import csv
fields = []
for i in llrVal:
    fields += [float(i)]
with open('./data/LLRFTN2'+str(ebno)+'db.csv', mode='w') as csv_file:
    writer = csv.writer(csv_file, delimiter='\n', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(fields)

In [ ]:
# [ 0.1166, 0.0046, 4.0003e-06]